In [ ]:
import numpy as np
import SimpleITK as sitk
import itk
import os
from scipy import signal
from skimage.transform import resize

In [ ]:
def get_list_of_files(base_dir):

    list_of_lists = []
    patients = os.listdir(base_dir)
    for p in patients:
        if p.startswith("mr_train_") and p.endswith("_label.nii.gz"):
            list_of_lists.append(os.path.join(base_dir, p))
    print("Found %d patients" % len(list_of_lists))
    return list_of_lists

def load_img(dir_img):
    # load SimpleITK Image
    img_sitk = sitk.ReadImage(dir_img)
    
    print("Pixel Type    {}".format(img_sitk.GetPixelID()))
    print("Size          {}".format(img_sitk.GetSize()))
    print("Origin        {}".format(img_sitk.GetOrigin()))
    print("Spacing       {}".format(img_sitk.GetSpacing()))
    print("Direction     {}".format(img_sitk.GetDirection()))

    # get pixel arrays from SimpleITK images
    img_npy = sitk.GetArrayFromImage(img_sitk)

    # get some metadata
    spacing = img_sitk.GetSpacing()
    # the spacing returned by SimpleITK is in inverse order relative to the numpy array we receive. If we wanted to
    # resample the data and if the spacing was not isotropic (in BraTS all cases have already been resampled to 1x1x1mm
    # by the organizers) then we need to pay attention here. Therefore we bring the spacing into the correct order so
    # that spacing[0] actually corresponds to the spacing of the first axis of the numpy array
    spacing = np.array(spacing)[::-1]
    direction = img_sitk.GetDirection()
    origin = img_sitk.GetOrigin()

    original_shape = img_npy.shape
    
    metadata = {
    'spacing': spacing,
    'direction': direction,
    'origin': origin,
    'original_shape': original_shape
    }
    
    return img_sitk, img_npy, metadata  


def reorient_to_rai(image):
    filter = itk.OrientImageFilter.New(image)
    filter.UseImageDirectionOn()
    filter.SetInput(image)
    m = itk.Matrix[itk.D, 3, 3]()
    m.SetIdentity()
    filter.SetDesiredCoordinateDirection(m)
    filter.Update()
    return filter.GetOutput()

   


def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkCosineWindowedSinc
    default_value = 100.0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)    



def affine_scale(transform, x_scale=3.0, y_scale=0.7):
    new_transform = sitk.AffineTransform(transform)
    print(np.array(transform.GetMatrix()).shape)
    matrix = np.array(transform.GetMatrix()).reshape((3,3))
    matrix[0,0] = x_scale
    matrix[1,1] = y_scale
    new_transform.SetMatrix(matrix.ravel())
    resampled = resample(img_sitk, new_transform)
    return resampled


def save_as_nii(img_npy, img_name, metadata):
    sitk_image = sitk.GetImageFromArray(img_npy)
    sitk_image.SetDirection(metadata['direction'])
    sitk_image.SetOrigin(metadata['origin'])
    # remember to revert spacing back to sitk order again
    sitk_image.SetSpacing(tuple(metadata['spacing'][[1, 2, 0]]))
    sitk.WriteImage(sitk_image, img_name)      

In [ ]:
base_dir = '/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr'
list_of_files = get_list_of_files(base_dir)
dir_img = list_of_files[19]
print(dir_img)

In [ ]:
img_sitk, img_npy, metadata = load_img(dir_img)

In [ ]:
def affine_rotate(transform, degrees=45.0):
    parameters = np.array(transform.GetParameters())
    new_transform = sitk.AffineTransform(transform)
    print(np.array(transform.GetMatrix()))
    matrix = np.array(transform.GetMatrix()).reshape((3,3))
    radians = -np.pi * degrees / 180.
    rotation = np.array([[np.cos(radians), -np.sin(radians), 0], 
                         [np.sin(radians), np.cos(radians), 0], 
                         [0, 0, 1]])  # https://stackoverflow.com/questions/9187387/3d-rotation-on-image
    new_matrix = np.dot(rotation, matrix)
    new_transform.SetMatrix(new_matrix.ravel())
    resampled = resample(img_sitk, new_transform)
    return resampled
    
affine = sitk.AffineTransform(3)

resu = affine_rotate(affine)

In [ ]:
print(type(resu))
sitk.WriteImage(resu, 'label_affine_' + list_of_files[19].split('/')[11:][0][0:-7] + '.nii.gz', True) 

In [ ]:
new_img_npy = signal.resample(img_npy, 16, axis=0)
img_resized = np.array([resize(i, (350, 350), anti_aliasing=True) for i in new_img_npy]).astype(np.float32)
print(img_npy.shape)
print(img_resized.shape)
save_as_nii(img_resized, 'modified_' + list_of_files[19].split('/')[11:][0][0:-7] + '.nii.gz', metadata)

In [ ]:
image = itk.imread(dir_img)
reoriented = reorient_to_rai(image)
itk.imwrite(reoriented, 'reoriented_' + list_of_files[19].split('/')[11:][0][0:-7] + '.nii.gz')